In [3]:
!pip install pymongo


   ---------------------------------------- 0.0/903.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/903.3 kB ? eta -:--:--
   ---------------------------------- ----- 786.4/903.3 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 903.3/903.3 kB 2.9 MB/s eta 0:00:00


In [2]:
import os
from dotenv import load_dotenv
import discord
import random
from discord.ext import commands
import nest_asyncio
from itertools import product
from collections import defaultdict
import json
import re
import pymongo
from pymongo import MongoClient


In [4]:
from dotenv import load_dotenv
load_dotenv()


False

In [8]:
import discord
from discord.ext import commands
from dotenv import load_dotenv
from pymongo import MongoClient
from collections import defaultdict
from itertools import product
import random
import json
import re
import nest_asyncio
import asyncio

# =============== CONFIGURATION ===============
CONFIG_FILE = "config.json"
ruined = False

def load_channel_id():
    try:
        with open(CONFIG_FILE, "r") as f:
            return json.load(f).get("allowed_channel_id")
    except FileNotFoundError:
        return None

def save_channel_id(channel_id):
    with open(CONFIG_FILE, "w") as f:
        json.dump({"allowed_channel_id": channel_id}, f)

# =============== BOT SETUP ===============
intents = discord.Intents.default()
intents.message_content = True
intents.emojis = True
intents.guilds = True
bot = commands.Bot(command_prefix='+', intents=intents)

nest_asyncio.apply()

client = MongoClient(MONGO_URI)
db = client['discord_bot']
scores_collection = db['user_scores']
config_collection = db['config']

current_index = 0
last_user_id = None
user_scores = defaultdict(int)

# =============== ALPHABET SEQUENCE ===============
def generate_alpha_sequence():
    length = 1
    while True:
        for combo in product("ABCDEFGHIJKLMNOPQRSTUVWXYZ", repeat=length):
            yield ''.join(combo)
        length += 1

alpha_gen = generate_alpha_sequence()
alpha_sequence = []

def get_sequence(n):
    while len(alpha_sequence) <= n:
        alpha_sequence.append(next(alpha_gen))
    return alpha_sequence[n]

def update_user_score(user_id):
    scores_collection.update_one(
        {"_id": user_id},
        {"$inc": {"score": 1}},
        upsert=True
    )

def get_user_score(user_id):
    doc = scores_collection.find_one({"_id": user_id})
    return doc['score'] if doc else 0

def get_leaderboard(limit=10):
    return list(scores_collection.find().sort("score", -1).limit(limit))

# =============== MONITOR TASK ===============
async def monitor_channel():
    await bot.wait_until_ready()
    channel_id = load_channel_id()
    if not channel_id:
        return

    counting_channel = bot.get_channel(channel_id)
    global current_index, last_user_id, ruined

    while not bot.is_closed():
        try:
            history = counting_channel.history(limit=1)
            last_msg = await anext(history, None)
            if last_msg is None or last_msg.author.bot or ruined:
                await asyncio.sleep(1)
                continue

            content = last_msg.content.strip().upper()

            for i in range(current_index, current_index + 3):  # allow slight catching up
                if get_sequence(i) == content:
                    current_index = i + 1
                    last_user_id = last_msg.author.id
                    break

        except Exception as e:
            print(f"[Monitor Error] {e}")

        await asyncio.sleep(1)

# =============== BOT EVENTS ===============
@bot.event
async def on_ready():
    print(f"✅ Logged in as {bot.user}")
    bot.loop.create_task(monitor_channel())

@bot.event
async def on_message(message):
    await bot.process_commands(message)
    global current_index, last_user_id, ruined

    if message.author.bot:
        return

    if re.search(r'<a?:\w+:\d+>', message.content):
        return

    if message.mentions or message.role_mentions or message.channel_mentions:
        return

    allowed_channel_id = load_channel_id()
    if allowed_channel_id is None or message.channel.id != allowed_channel_id:
        return

    content = message.content.strip()

    if content.startswith(('+', '-')):
        return

    content = content.upper()

    # If game is ruined, only resume when 'A' is typed
    if ruined:
        if content == "A":
            current_index = 1
            last_user_id = message.author.id
            user_scores[message.author.id] += 1
            update_user_score(message.author.id)
            emoji = random.choice(message.guild.emojis)
            await message.add_reaction(emoji)
            ruined = False
        else:
            return  # Ignore everything until "A"
        return

    expected = get_sequence(current_index)

    if content == expected:
        if message.author.id == last_user_id:
            await message.add_reaction("❌")
            await message.channel.send("Fuck off loner")
        else:
            emoji = random.choice(message.guild.emojis)
            await message.add_reaction(emoji)
            current_index += 1
            last_user_id = message.author.id
            user_scores[message.author.id] += 1
            update_user_score(message.author.id)
    else:
        await message.add_reaction("❌")
        await message.channel.send("You ruined it asshole 😡 next alphabet is A")
        current_index = 0
        last_user_id = None
        ruined = True

@bot.event
async def on_message_edit(_, after):
    global current_index, last_user_id
    await after.clear_reactions()
    await after.add_reaction("❌")
    await after.channel.send("You ruined it asshole 😡 next alphabet is A (editing won't save you)")
    current_index = 0
    last_user_id = None

# =============== ADMIN COMMANDS ===============
@bot.command(name='setchannel')
@commands.has_permissions(administrator=True)
async def setchannel(ctx):
    channel_id = ctx.channel.id
    save_channel_id(channel_id)
    await ctx.send(f"✅ This channel (`#{ctx.channel.name}`) has been set as the allowed counting channel.")

# =============== USER COMMANDS ===============
@bot.command(name="leaderboard")
async def leaderboard(ctx):
    leaderboard_data = get_leaderboard()
    if not leaderboard_data:
        await ctx.send("No one has scored yet!")
        return

    description = ""
    for i, entry in enumerate(leaderboard_data, start=1):
        user = await bot.fetch_user(entry['_id'])
        description += f"{i}. **{user.name}** — {entry['score']} ✅\n"

    embed = discord.Embed(title="🏆 Leaderboard", description=description, color=0x00ff00)
    await ctx.send(embed=embed)

@bot.command(name="mystats")
async def mystats(ctx, member: discord.Member = None):
    if member is None:
        member = ctx.author
    score = get_user_score(member.id)
    await ctx.send(f"👤 **{member.name}**, you have **{score}** correct counts ✅.")

# =============== RUN BOT ===============
bot.run(TOKEN)


[2025-06-18 18:43:52] [INFO    ] discord.client: logging in using static token
[2025-06-18 18:43:55] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: bc888996db9f9443f38d1c2286d381dc).


✅ Logged in as AlphaCount#4179
